# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Shared Working Space rental business in Izmir, Turkey._**
- Build a dataframe of neighborhoods in Izmir, Turkey by web scraping the data from https://www.atlasbig.com/tr/izmirin-ilceleri
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Shared Working Space rental business
***

### 1. Import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2. Scrap the Data

In [2]:
url = "https://www.atlasbig.com/tr/izmirin-ilceleri"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'lxml')

In [3]:
My_table = soup.find_all('td')
My_table

[<td>
 <a href="/tr/izmir-buca">Buca</a>
 </td>,
 <td>			
 																	492,252
 															</td>,
 <td>			
 													
 																			173 
 																								</td>,
 <td>
 <a href="/tr/izmir-karabaglar">Karabağlar</a>
 </td>,
 <td>			
 																	480,790
 															</td>,
 <td>			
 													
 																			92 
 																								</td>,
 <td>
 <a href="/tr/izmir-bornova">Bornova</a>
 </td>,
 <td>			
 																	442,839
 															</td>,
 <td>			
 													
 																			220 
 																								</td>,
 <td>
 <a href="/tr/izmir-konak">Konak</a>
 </td>,
 <td>			
 																	363,181
 															</td>,
 <td>			
 													
 																			25 
 																								</td>,
 <td>
 <a href="/tr/izmir-karsiyaka">Karşıyaka</a>
 </td>,
 <td>			
 																	342,062
 															</td>,
 <td>			
 													
 																			57 
 																								</td>,
 <td>
 <a href="/tr/izmir-bayrakl

Store the scraped data into a pandas dataframe.

In [4]:
res=[]
for tr in My_table:
    td = tr.find_all('a')
    row = [td.text.strip() for td in td if td.text.strip()]
    
    if row:
        res.append(row)
df_izmir=pd.DataFrame(res,columns=["District"])
df_izmir

,District
0,Buca
1,Karabağlar
2,Bornova
3,Konak
4,Karşıyaka
5,Bayraklı
6,Çiğli
7,Torbalı
8,Menemen
9,Gaziemir


### 3. Get the Geographical Coordinates of the Neighborhoods.

In [5]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Foça, İzmir")
location.longitude

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=5)
df_izmir['location'] = df_izmir['District'].apply(geocode)
df_izmir['point'] = df_izmir['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# split point column into latitude, longitude and altitude columns
df_izmir[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_izmir['point'].tolist(), index=df_izmir.index)

In [ ]:
df_izmir

### 4. Display the Neighborhoods on a Map

In [ ]:
address = 'İzmir'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of İzmir, Turkiye are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_izmir = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, neighborhood in zip(df_izmir['latitude'], df_izmir['longitude'], df_izmir['District']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_izmir)  
    
map_izmir

### 5. Use the Foursquare API to explore the neighborhoods

Set up our credentials to make requests to Foursquare API

In [6]:
CLIENT_ID = '31VAXHFA20350LL2SV1BD1INTYMXX3EEM3U2NKVZ5HIRQBBS' # your Foursquare ID
CLIENT_SECRET = '3QKIZATPCAI0QI14MMBXYZW52KXZ5LARAW1O2SCKXBJTIS2B' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 31VAXHFA20350LL2SV1BD1INTYMXX3EEM3U2NKVZ5HIRQBBS
CLIENT_SECRET:3QKIZATPCAI0QI14MMBXYZW52KXZ5LARAW1O2SCKXBJTIS2B


Let's fetch the latitude and longitude of the first neighborhood and get the venues inside it as a test.

In [ ]:
neighborhood_latitude = df_izmir.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_izmir.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df_izmir.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 4500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
izmir_venues = getNearbyVenues(names=df_izmir['District'],
                                   latitudes=df_izmir['latitude'],
                                   longitudes=df_izmir['longitude']
                                  )

In [ ]:
izmir_venues.head()

In [ ]:
izmir_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
izmir_onehot = pd.get_dummies(izmir_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
izmir_onehot['Neighborhood'] = izmir_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [izmir_onehot.columns[izmir_onehot.columns.get_loc('Neighborhood')]] + list(izmir_onehot.columns[:-1])
izmir_onehot = izmir_onehot[fixed_columns]
izmir_onehot = izmir_onehot.loc[:,~izmir_onehot.columns.duplicated()]
izmir_onehot.head()

In [ ]:
izmir_grouped = izmir_onehot.groupby('Neighborhood').mean().reset_index()
print(len(izmir_grouped[izmir_grouped["Park"] > 0]) )
print(len(izmir_grouped[izmir_grouped["Café"] > 0]))

In [ ]:
num_top_venues = 5

for hood in izmir_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = izmir_grouped[izmir_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = izmir_grouped['Neighborhood']

for ind in np.arange(izmir_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(izmir_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
izmir_test=neighborhoods_venues_sorted[(neighborhoods_venues_sorted['1st Most Common Venue']=='Café') | (neighborhoods_venues_sorted['1st Most Common Venue']=='Restaurant')]